# Deterministic Event Driven Simulation

Add the package:
```julia
add "https://github.com/JuDO-dev/AirBorne.jl#dev"
```

## Fetch data

In [1]:
# Get Data
using AirBorne: AirBorne
using Dates: Dates, datetime2unix, DateTime, Day
using Logging
using DotMaps:DotMap

In [170]:

# Get 5 Years of data for APPL
from = Dates.DateTime("2017-01-01")
to = Dates.DateTime("2022-01-01")
u_from = string(round(Int, Dates.datetime2unix(from)))
u_to = string(round(Int, Dates.datetime2unix(to)))
data = AirBorne.ETL.YFinance.get_interday_data(["AAPL","GOOG"], u_from, u_to)

# println(size(r))
data[!,:close]=float.(data[!,:close])
data[!,:high]=float.(data[!,:high])
data[!,:low]=float.(data[!,:low])
data[!,:open]=float.(data[!,:open])
data[!,:unix]=Int.(data[!,:unix])
data[!,:volume]=Int.(data[!,:volume])

AirBorne.ETL.Cache.store_bundle(data; bundle_id="demo", archive=true) 
# data

[ Info: Storing /root/project/.AirBorneCache/demo/2023_06_11_20_26_37_681.parq.snappy


✏ Parquet2.FileWriter{IOStream}(/root/project/.AirBorneCache/demo/2023_06_11_20_26_37_681.parq.snappy)

In [2]:
data= AirBorne.ETL.Cache.load_bundle("demo") 
# const cdata = data # Making it constant doesn't seem to improve speed significantly, at least not in small datasets
# @time data[1:1000,:]
# @time cdata[1:1000,:]
# @time data[100:end,:]
# @time cdata[100:end,:]

Row,close,high,low,open,volume,date,unix,exchangeName,timezone,currency,symbol
,Float64,Float64,Float64,Float64,Int64,DateTime,Int64,String,String,String,String
1,29.0375,29.0825,28.69,28.95,115127600,2017-01-03T10:30:00,1483453800,NMS,America/New_York,USD,AAPL
2,29.005,29.1275,28.9375,28.9625,84472400,2017-01-04T10:30:00,1483540200,NMS,America/New_York,USD,AAPL
3,29.1525,29.215,28.9525,28.98,88774400,2017-01-05T10:30:00,1483626600,NMS,America/New_York,USD,AAPL
4,29.4775,29.54,29.1175,29.195,127007600,2017-01-06T10:30:00,1483713000,NMS,America/New_York,USD,AAPL
5,29.7475,29.8575,29.485,29.4875,134247600,2017-01-09T10:30:00,1483972200,NMS,America/New_York,USD,AAPL
6,29.7775,29.845,29.575,29.6925,97848400,2017-01-10T10:30:00,1484058600,NMS,America/New_York,USD,AAPL
7,29.9375,29.9825,29.65,29.685,110354400,2017-01-11T10:30:00,1484145000,NMS,America/New_York,USD,AAPL
8,29.8125,29.825,29.5525,29.725,108344800,2017-01-12T10:30:00,1484231400,NMS,America/New_York,USD,AAPL
9,29.76,29.905,29.7025,29.7775,104447600,2017-01-13T10:30:00,1484317800,NMS,America/New_York,USD,AAPL


### Data Flow in Simulation

I want to the user to be able to place orders and schedule events, however I do not want the user to be able to add items to its portfolio directly, that is something that only the market can do.

But I want the user to be able to see its own portfolio or have access to it. This is not possible in Julia. Some workarounds can be put in place as checking the size of the variable or passing a deepcopy can be done, but they carry extra computational cost. 

Since is  in the best interest of the user to use the tool properly the  solution for now is just to make it clear in the documentation what each thing  is and how are meant to be used

```julia
Base.summarysize(x)# Returns the  size of a variable
```

### Cool Julia Feature
Recently I discovered that the generation of immutable structures do not need memory allocation. This is because they have fixed sizes and the variables are stored in fast temporary  memory positions.[Julia Notes on Static Arrays](https://m3g.github.io/JuliaNotes.jl/stable/immutable/#Static-arrays)


In [4]:
# function nms_execute_orders!(orders,portfolio,from,to,data)
#     return "Orders Executed"
# end
# ListOfMarkets = Dict("NMS"=>nyse_execute_orders!)

# AAPL is traded in NASDAQ Global Select Market.

# const ListOfMarkets = {"NYSE"=>nyse_execute_orders}


# Experimental Prototype

In [5]:
# Recommendation Once I figures
struct Order
    market::String
    specs::DotMap
end
# Order("NYSE",DotMap(Dict("amount"=>100,"asset"=>"AAPL")))

In [15]:
# Utility Functions
using Pipe: @pipe
using DataFrames: groupby,combine

function sortedStructInsert!(v::Vector, x ,sort_symbol; rev=true) 
    (splice!(v, searchsorted(v,x,by= v->getproperty(v,sort_symbol), rev=rev), [x]); v)
end

function sortStruct!(v::Vector, symbol;rev=true) 
    sort!(v, by = v->getproperty(v,symbol), rev=rev) 
end

function get_latest(df,id_symbols,sort_symbol)
    return combine(groupby(df, id_symbols)) do sdf; sdf[argmax(sdf[!,sort_symbol]), :]; end    
end

"""
    get_latest_N(df,id_symbols,sort_symbol,N)

Retrieves last N records from a dataframe, sortying by sort_symbol and grouping by id_symbols.

```julia
get_latest_N(past_data,[:exchangeName,:symbol],:date,2)
```
"""
function get_latest_N(df,id_symbols,sort_symbol,N)
    return @pipe df |>
        groupby(_, id_symbols) |>
        combine(_) do sdf
            sorted = sort(sdf, sort_symbol)
            first(sorted, N)
        end
end
# Testing do capabilities
# function add(a,b)
#     return a+b
# end
# add(3) do x

get_latest_N

In [22]:
struct TimeEvent
    date::Dates.DateTime
    type::String
end
function lenght(::TimeEvent)
    return 1
end

lenght (generic function with 1 method)

In [17]:
function place_order!(context,order)
    push!(context.activeOrders,order)
end

function initialize!(context)
    
    ####################################
    ####  Specify Account Balance  #####
    ####################################
    context.accounts.usd=DotMap(Dict())
    context.accounts.usd.balance=100000
    context.accounts.usd.currency="USD"
    
    #############################
    ####  Specify next event  ###
    #############################
    next_event_date = context.current_event.date + Day(1) 
    new_event = TimeEvent(next_event_date,"data_transfer")
    sortedStructInsert!(context.eventList,new_event,:date)
end




trading_logic! (generic function with 1 method)

In [81]:
# Market Functions
using DotMaps: keys
"""
    expose_data(context,data)

    This function determine how the data is transformed and filtered before being passed to the user.

"""
function expose_data(context,data)
    return available_data(context,data)
end

"""
    available_data(context,data)

    This function determine how the data is transformed and filtered before being passed to the user.

"""
function available_data(context,data)
    return data[data.date .<= context.current_event.date,:]
end


function addSecurityToPortfolio(portfolio::Vector{Any},security::DotMap)
    push!(portfolio,security)
end

        # security=DotMap(Dict("exchangeName"=>order.market,"ticker"=>order.specs.ticker,"shares"=>shares))
function addSecurityToPortfolio(portfolio::Union{DotMap,Dict},security::Union{DotMap,Dict})
    key=get(security,"exchangeName","MISSING") * "/" * get(security,"ticker","MISSING")
    if !(haskey(portfolio,key)) # A try catch approach may be more performant
        portfolio[key]=0
    end
    portfolio[key] += get(security,"shares",nothing)
end

function addJournalEntryToLedger(ledger::Vector{Any},journalEntry::Union{DotMap,Dict})
    push!(ledger,journalEntry)
end

"""
    execute_orders(from, to, context,data)

This function updates the portfolio of the user that is stored in the variable context
"""
function execute_orders!(from, to, context,data)
    # @info "Executing orders: $from - $to"
    # @info "Orders: $(context.activeOrders)"
    # Assume that the order is placed the date before and
    cur_data = get_latest(available_data(context,data),[:exchangeName,:symbol],:date)
    incomplete_orders = Vector{Any}([])
    
    while length(context.activeOrders)>0
        order= pop!(context.activeOrders)
        # @info order
        success=true
        if order.specs.type=="MarketOrder"
        # Transaction data
        price = cur_data[cur_data.symbol.==order.specs.ticker,:open][1]
        shares = order.specs.shares
        # @info price
        # @info shares
        transaction_amount = price * order.specs.shares
        # @info transaction_amount

        # Partial order corrections
        if (transaction_amount>=order.specs.account.balance) && (shares>0) # If not enough money to buy execute partially
            success = false
            transaction_amount = order.specs.account.balance
            shares = transaction_amount / price
            # Here there should be some logic to allow/forbid fractional transactions
        end
                
        if shares==0 # Skip steps below if there are no 
            continue
        end
        # @info "Funds available: $(order.specs.account.balance)"
        # @info "Transaction amount: $(shares)x$(price)=$(transaction_amount)"
        order.specs.account.balance -= transaction_amount
        # @info "Funds remaining: $(order.specs.account.balance)"
        
        # Form Security
        security=Dict("exchangeName"=>order.market,"ticker"=>order.specs.ticker,"shares"=>shares,"price"=>price)
        
        journal_entry=deepcopy(security)
        journal_entry["price"]=price
        journal_entry["amount"]=transaction_amount
        journal_entry["date"]=deepcopy(to) # Improve the logic of determining when a transaction takes place
        # I.e. one could define an open and close times for a market and use that instead.
                
        addJournalEntryToLedger(context.ledger,journal_entry)
        addSecurityToPortfolio(context.portfolio,security)
                
        elseif order.specs.type=="LimitOrder"
            @info "LimitOrder has not yet been implemented, please use MarketOrder"
        end
            
        # Incomplete orders are to be put back
        if !(success)
            order.specs.shares -= shares # Reduce the amount of shares
            push!(incomplete_orders,order)
        end
    end
    append!(context.activeOrders,incomplete_orders)
end
        
"""
    deepPush!(list,element)

        Inserts the deepcopy of an element into a collection 
"""
function deepPush!(list,element)
    push!(list,deepcopy(element))
end


deepPush!

# Run Simulation


In [82]:
# dates = from:Day(1):to 
# function run(data::DataFrame, initialize!::Function, trading_logic!::Function, execute_orders!::Function,expose_data::Function;audit=true)
    
   # events = [ TimeEvent(date,"data_release") for date in data[!,"date"]] 
   # context=Context(events,[],[])

# Context definition
function DM()
    return DotMap(Dict())
end

mutable struct Context
   eventList::Vector{TimeEvent} # List of events defined by the user
   activeOrders::Vector{Order} # List of orders
   current_event::TimeEvent
   portfolio::Dict # The market determines the portfolio
   accounts::DotMap # The market determines the account
   ledger::Vector{Any} # List of transactions
   audit::DotMap
   extra::DotMap 
end

audit=true 

context = Context([],[],TimeEvent(findmin(data.date)[1],"start"),Dict(),DM(),[],DM(),DM())

HiddenContext=DotMap(Dict())
HiddenContext.portfolioHistory=[]
HiddenContext.accountHistory=[]
HiddenContext.eventHistory=[]
HiddenContext.extraHistory=[] 


initialize!(context) 
if audit
    deepPush!(HiddenContext.portfolioHistory,context.portfolio)
    deepPush!(HiddenContext.accountHistory,context.accounts)
    deepPush!(HiddenContext.eventHistory,context.current_event)
    deepPush!(HiddenContext.extraHistory,context.extra)
end
counter = 0
max_iter= 50
while ((size(context.eventList)[1]>0)) && (counter<max_iter)
    counter+=1
    
    past_event_date=context.current_event.date
    #############################
    ####   Pick next event   ####
    #############################
    # Sort Event List 
    sortStruct!(context.eventList,:date)
    context.current_event=pop!(context.eventList)# Removes the last item from the collection
    
    #############################################
    ####   Execute orders since last event   ####
    #############################################
    execute_orders!(past_event_date, context.current_event.date, context, data)
    if audit
        deepPush!(HiddenContext.portfolioHistory,context.portfolio)
        deepPush!(HiddenContext.accountHistory,context.accounts)
        deepPush!(HiddenContext.eventHistory,context.current_event)
        deepPush!(HiddenContext.extraHistory,context.extra)
    end
    #########################
    ####   Expose Data   ####
    #########################
    exposed_data = expose_data(context,data) # Mechanism for which the market exposes the data to the user
    
    
    ########################################################
    ####   Let user process data and place new orders   ####
    ########################################################
    trading_logic!(context,exposed_data)
    
end
if audit
    context.audit=HiddenContext
end
# return context
# end

DotMap(Dict{Symbol, Any}(:accountHistory => Any[DotMap(Dict{Symbol, Any}(:usd => DotMap(Dict{Symbol, Any}(:currency => "USD", :balance => 100000)))), DotMap(Dict{Symbol, Any}(:usd => DotMap(Dict{Symbol, Any}(:currency => "USD", :balance => 100000)))), DotMap(Dict{Symbol, Any}(:usd => DotMap(Dict{Symbol, Any}(:currency => "USD", :balance => 97102.00004577637)))), DotMap(Dict{Symbol, Any}(:usd => DotMap(Dict{Symbol, Any}(:currency => "USD", :balance => 94182.50007629395)))), DotMap(Dict{Symbol, Any}(:usd => DotMap(Dict{Symbol, Any}(:currency => "USD", :balance => 91263.00010681152)))), DotMap(Dict{Symbol, Any}(:usd => DotMap(Dict{Symbol, Any}(:currency => "USD", :balance => 88343.5001373291)))), DotMap(Dict{Symbol, Any}(:usd => DotMap(Dict{Symbol, Any}(:currency => "USD", :balance => 85394.75021362305)))), DotMap(Dict{Symbol, Any}(:usd => DotMap(Dict{Symbol, Any}(:currency => "USD", :balance => 82425.50029754639)))), DotMap(Dict{Symbol, Any}(:usd => DotMap(Dict{Symbol, Any}(:currency => 

In [91]:
using DataFrames: DataFrame, Not, select!

function parse_accountHistory(accountHistory) # Probably I will also need the value of the stock at the audit point in time.
    accounts = Set([ key  for p in accountHistory for key in keys(p)])
    N=length(accountHistory)
    baseDf = DataFrame(ix=1:N)#, C=1:500)
    for account=accounts 
            baseDf[!,account]=[ get(get(a,account,nothing),"balance",nothing)  for a in accountHistory]
    end
    return select!(baseDf, Not([:ix]))
end


function parse_portfolioHistory(portfolioHistory) # Probably I will also need the value of the stock at the audit point in time.
    assetIds = Set([ key  for p in portfolioHistory for key in keys(p)])
    N=length(portfolioHistory)
    baseDf = DataFrame(ix=1:N)#, C=1:500)
    for asset=assetIds 
        baseDf[!,asset]=[ get(p,asset,nothing)  for p in portfolioHistory]
    end
    return select!(baseDf, Not([:ix]))
end


# Beautiful
# dfPortfolio=parse_portfolioHistory( context.audit.portfolioHistory)
# dfPortfolio=parse_accountHistory(context.audit.accountHistory)
accountHistory=context.audit.accountHistory
accounts = Set([ key  for p in accountHistory for key in keys(p)])
N=length(accountHistory)

baseDf = DataFrame(ix=1:N)#, C=1:500)
for account=accounts 
        baseDf[!,account]=[ get(get(a,account,nothing),"balance",nothing)  for a in accountHistory]
end

In [92]:
baseDf
# accountHistory

Row,ix,usd
,Int64,Real
1,1,100000
2,2,100000
3,3,97102.0
4,4,94182.5
5,5,91263.0
6,6,88343.5
7,7,85394.8
8,8,82425.5
9,9,79457.0


In [93]:
DM2()= DotMap(Dict())

DM2 (generic function with 1 method)

In [252]:
typeof(context.portfolio)

Vector{Any} (alias for Array{Any, 1})

In [66]:

# Call Initialization function
initialize!(context)

[ Info: Any[]
[ Info: 2017-01-04T10:30:00


# Currencies: JuliaFinance

``` julia
# usd = Finance.Currencies.currency(:USD)
# @info "Symbol: $(Finance.Currencies.symbol(usd))"
# @info "Name: $(Finance.Currencies.name(usd))"
# @info "Code: $(Finance.Currencies.code(usd))"
# @info "Minor Unit: $(Finance.Currencies.unit(usd))"

# jpy = Finance.Currencies.currency(:JPY)
# @info "Symbol: $(Finance.Currencies.symbol(jpy))"
# @info "Name: $(Finance.Currencies.name(jpy))"
# @info "Code: $(Finance.Currencies.code(jpy))"
# @info "Minor Unit: $(Finance.Currencies.unit(jpy))"

# Assets seem not to work
# using Assets
# @cash USD
# Finance.Assets.cash
```

In [77]:

module Finance
export Currencies
export Assets, @cash
using Currencies: Currencies
using Assets: Assets, @cash
end



LoadError: UndefVarError: @cash not defined

# Experiments on immutability

In [19]:
# context.marketName="2" # Fails if is immutable. Good
# const market="NYSE" # Constant values cannot be modified
# market=2
# const 2

# b =  Base.ImmutableDict("b"=>2,"c"=>3)
# g=[2,3,4]
# c =  Base.ImmutableDict("b"=>g,"c"=>3)
# b["c"]=2
# setindex!(b, "b", 3)
# b

# Test from package

In [5]:
using AirBorne: AirBorne
using AirBorne.ETL.Cache: load_bundle, store_bundle
using AirBorne.Engines.DEDS: run
using AirBorne.Markets.StaticMarket: execute_orders!, expose_data, parse_portfolioHistory, parse_accountHistory
using Dates: Dates
include("./AlwaysBuyStrategy.jl")
cache_dir = joinpath(@__DIR__, "assets", "cache")
# To generate this data use:
from = Dates.DateTime("2017-01-01"); to = Dates.DateTime("2022-01-01")
u_from = string(round(Int, Dates.datetime2unix(from))); u_to = string(round(Int, Dates.datetime2unix(to)))
data = AirBorne.ETL.YFinance.get_interday_data(["AAPL","GOOG"], u_from, u_to)

data[!,:close]=float.(data[!,:close])
data[!,:high]=float.(data[!,:high])
data[!,:low]=float.(data[!,:low])
data[!,:open]=float.(data[!,:open])
data[!,:unix]=Int.(data[!,:unix])
data[!,:volume]=Int.(data[!,:volume])

AirBorne.ETL.Cache.store_bundle(data; bundle_id="demo", archive=true) 
    # data = load_bundle("demo"; cache_dir=cache_dir)
initialize! = AlwaysBuyStrategy.initialize!
trading_logic! = AlwaysBuyStrategy.trading_logic!
max_iter = 50
results = run(
    data,
    initialize!,
    trading_logic!,
    execute_orders!,
    expose_data;
    audit=true,
    max_iter=max_iter,
)
parsed_portfolioHistory = parse_portfolioHistory(results.audit.portfolioHistory)
parsed_accountHistory   = parse_accountHistory(results.audit.accountHistory)

[ Info: Storing /root/project/.AirBorneCache/demo/2023_06_22_12_30_43_121.parq.snappy


Row,usd
,Real
1,100000
2,100000
3,97102.0
4,94182.5
5,91263.0
6,88343.5
7,85394.8
8,82425.5
9,79457.0


In [17]:
using DataFrames: DataFrame, hcat
eventDf=DataFrame(Dict("event"=>[ e.date for e in results.audit.eventHistory]))
hcat(eventDf,parsed_accountHistory,parsed_portfolioHistory)

Row,event,usd,NMS/AAPL
,DateTime,Real,Union…
1,2017-01-03T10:30:00,100000,
2,2017-01-04T10:30:00,100000,
3,2017-01-05T10:30:00,97102.0,100
4,2017-01-06T10:30:00,94182.5,200
5,2017-01-07T10:30:00,91263.0,300
6,2017-01-08T10:30:00,88343.5,400
7,2017-01-09T10:30:00,85394.8,500
8,2017-01-10T10:30:00,82425.5,600
9,2017-01-11T10:30:00,79457.0,700
